In [0]:
import os
from scipy.io import loadmat

# Get current notebook directory
notebook_dir = os.getcwd()
file_path = os.path.join(notebook_dir, "test.mat")

input = loadmat(file_path)
a = input["a"].flatten()
x = input["x"].flatten()

![assignement4_a.png](./assignement4_a.png "assignement4_a.png")


#### 1)

This is equal to the one step forecasting 